# Building Simple Models

In this tutorial we look at how to build a simple source model and sample the parameters from a variety of sources including pzflow.

## Parameterized Nodes

All sources of information in TDAstro live as `ParameterizedNode`s. This allows us to link the nodes (and their variables) together and sample them as a single block. As you will see in this tutorial, most of the nodes are specific to the object that you want to simulate. For example if we wanted to create a static source in the night sky with a brightness of 10, we could use: 

In [ ]:
from tdastro.sources.static_source import StaticSource

source = StaticSource(brightness=10.0, node_label="my_static_source")

`ParameterizedNode`s can then be sampled with the `sample_parameters()` function. This function will return a `GraphState` data structure that stores all of the data of the samples. 

**Note:** Users do not need to know the details of the `GraphState` storage, only that it can be accessed like a dictionary using the node's label and the variable name.

In [ ]:
state = source.sample_parameters(num_samples=10)
state["my_static_source"]["brightness"]

The sample function produced 10 independent samples of our system's state.

The brightness values of these samples are not particularly interesting because we were sampling from a fixed parameter. The brightness is always 10.0. However TDAstro allows the user to set a node's parameter from a variety of sources including constants (as with 10.0 above), the values stored in other nodes, or even the results of a "functional" or "computation" type node.

TDAStro includes the built-in `NumpyRandomFunc` which will sample from a given numpy function and use the results to set a given parameter.

In [ ]:
from tdastro.math_nodes.np_random import NumpyRandomFunc

brightness_func = NumpyRandomFunc("uniform", low=11.0, high=15.5)
source2 = StaticSource(brightness=brightness_func, node_label="my_static_source_2")
state = source2.sample_parameters(num_samples=10)

state["my_static_source_2"]["brightness"]

Now each of our 10 samples use different a different brightness value.

We can make the distributions of objects more interesting, by using combinations of randomly generated parameters. Note here that we do not need to create the random nodes separately (as with the line `brightness_func = ...`). 

As shown below, we will often include the random sampler definition directly in the arguments as the parameter value.

```
    brightness=NumpyRandomFunc("normal", loc=20.0, scale=2.0),
```

The sampling process create a vector of samples for each parameter such that the `i`-th value of each parameter is from the same sampling run. Again, the user will rarely (if ever) need to interact with the samples directly.

Here we sample the brightness from a Gaussian and sample the redshift from a uniform distribution.

In [ ]:
source3 = StaticSource(
    brightness=NumpyRandomFunc("normal", loc=20.0, scale=2.0),
    redshift=NumpyRandomFunc("uniform", low=0.1, high=0.5),
    node_label="test",
)

num_samples = 10
state = source3.sample_parameters(num_samples=num_samples)
for i in range(num_samples):
    print(f"{i}: brightness={state['test']['brightness'][i]} redshift={state['test']['redshift'][i]}")

Sample 0 consists of all the parameter values for that sample (everything at index=0), sample 1 consists of all parameter values for that sample (everything at index=1), and so forth. We can slice out a single sample using `extract_single_sample()` and display it. This is particularly important when different parameter values for a given sample depend on each other. We will see this case below when sampling a source's RA from it's host's RA.

In [ ]:
single_sample = state.extract_single_sample(0)
print(str(single_sample))

You'll notice that there are more parameters than we manually set. Parameters are created automatically by the nodes if needed. In particular functional nodes often create extra parameters for book keeping. In general the user should not need to worry about these extra parameters. They can access the ones of interest with the dictionary notation.

## Linked Sources

Often the values of one node might depend on the values of another. A great case of this is a source/host pair where the location of the source depends on that of the host. We can access another node’s sampled parameters using a `.` notation: `{model_object}.{parameter_name}`

In [ ]:
host = StaticSource(brightness=15.0, ra=1.0, dec=2.0, node_label="host")
source = StaticSource(brightness=10.0, ra=host.ra, dec=host.dec, node_label="source")
state = source.sample_parameters(num_samples=5)

for i in range(5):
    print(
        f"{i}: Host=({state['host']['ra'][i]}, {state['host']['dec'][i]})"
        f"Source=({state['source']['ra'][i]}, {state['source']['dec'][i]})"
    )

We can combine the node-parameter references with functional nodes to perform actions such as sampling with noise.

Here we generate the host's (RA, dec) from a uniform patch of the sky and then generate the source's (RA, dec) using a Gaussian distribution centered on the host's position. For each sample the host and source should be close, but not necessarily identical.

In [ ]:
host = StaticSource(
    brightness=15.0,
    ra=NumpyRandomFunc("uniform", low=10.0, high=15.0),
    dec=NumpyRandomFunc("uniform", low=-10.0, high=10.0),
    node_label="host",
)

source = StaticSource(
    brightness=100.0,
    ra=NumpyRandomFunc("normal", loc=host.ra, scale=0.1),
    dec=NumpyRandomFunc("normal", loc=host.dec, scale=0.1),
    node_label="source",
)
state = source.sample_parameters(num_samples=10)

import matplotlib.pyplot as plt

ax = plt.figure().add_subplot()
ax.plot(state["host"]["ra"], state["host"]["dec"], "b.")
ax.plot(state["source"]["ra"], state["source"]["dec"], "r.")

for i in range(5):
    print(
        f"{i}: Host=({state['host']['ra'][i]}, {state['host']['dec'][i]})    "
        f"Source=({state['source']['ra'][i]}, {state['source']['dec'][i]})"
    )

Again we can access all the information for a single sample. Here we see the full state tracked by the system. In addition to the `host` and `source` nodes we created, the information for the functional nodes is tracked.

In [ ]:
single_sample = state.extract_single_sample(0)
print(str(single_sample))

It is interesting to note that functional nodes themselves are parameterized nodes, allowing for more complex forms of chaining. For example we could set the `low` parameter from one of the `NumpyRandomFunc`s from another function node. This allows us to specify priors and comlex distributions.

## pzflow

We can use the `pzflow` package to generate data from joint distributions that have been learned from real data. Sampling with `pzflow` creates Pandas tables of values. We can access each column using the same `.` notation as above, allowing us to use a consistent set of values from multiple variables.

We start by loading a simple pzflow that we have saved with (nonsense) redshift and brightness values.

In [ ]:
from pzflow import Flow
from tdastro import _TDASTRO_TEST_DATA_DIR

pzflow = Flow(file=_TDASTRO_TEST_DATA_DIR / "test_flow.pkl")
pzflow.sample(nsamples=10)

We can create a `ParameterizedNode` for the pzflow as shown below. This simply wraps the existing `Flow` object as a `ParameterizedNode` and provides it a user readable name ("pznode") for later queries. Note that within the `PZFlowNode` node all parameters for the `i`-th sample are generated together.

In [ ]:
from tdastro.astro_utils.pzflow_node import PZFlowNode

pz_node = PZFlowNode(pzflow, node_label="pznode")

source = StaticSource(
    brightness=pz_node.brightness,
    redshift=pz_node.redshift,
    node_label="source",
)

num_samples = 10
state = source.sample_parameters(num_samples=num_samples)
for i in range(num_samples):
    print(f"{i}: brightness={state['source']['brightness'][i]}  z={state['source']['redshift'][i]}")